In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('rec-system').getOrCreate()

In [2]:
from pyspark.ml.recommendation import ALS

In [3]:
from pyspark.ml.evaluation import RegressionEvaluator

In [4]:
data = spark.read.csv('/FileStore/tables/movielens_ratings.csv', inferSchema=True, header=True)

In [5]:
data.show()

+-------+------+------+
movieId|rating|userId|
+-------+------+------+
 2| 3.0| 0|
 3| 1.0| 0|
 5| 2.0| 0|
 9| 4.0| 0|
 11| 1.0| 0|
 12| 2.0| 0|
 15| 1.0| 0|
 17| 1.0| 0|
 19| 1.0| 0|
 21| 1.0| 0|
 23| 1.0| 0|
 26| 3.0| 0|
 27| 1.0| 0|
 28| 1.0| 0|
 29| 1.0| 0|
 30| 1.0| 0|
 31| 1.0| 0|
 34| 1.0| 0|
 37| 1.0| 0|
 41| 2.0| 0|
+-------+------+------+
only showing top 20 rows

In [6]:
data.select('userId').groupby('userId').count().sort('userId').show(100)

+------+-----+
userId|count|
+------+-----+
 0| 49|
 1| 49|
 2| 46|
 3| 48|
 4| 55|
 5| 49|
 6| 57|
 7| 54|
 8| 49|
 9| 53|
 10| 44|
 11| 56|
 12| 55|
 13| 48|
 14| 57|
 15| 48|
 16| 45|
 17| 46|
 18| 52|
 19| 49|
 20| 47|
 21| 48|
 22| 56|
 23| 52|
 24| 52|
 25| 46|
 26| 49|
 27| 46|
 28| 50|
 29| 46|
+------+-----+

In [7]:
data.select('userId').groupby('userId').count().show(100)

+------+-----+
userId|count|
+------+-----+
 28| 50|
 26| 49|
 27| 46|
 12| 55|
 22| 56|
 1| 49|
 13| 48|
 6| 57|
 16| 45|
 3| 48|
 20| 47|
 5| 49|
 19| 49|
 15| 48|
 9| 53|
 17| 46|
 4| 55|
 8| 49|
 23| 52|
 7| 54|
 10| 44|
 25| 46|
 24| 52|
 29| 46|
 21| 48|
 11| 56|
 14| 57|
 2| 46|
 0| 49|
 18| 52|
+------+-----+

In [8]:
data.describe().show()

+-------+------------------+------------------+------------------+
summary| movieId| rating| userId|
+-------+------------------+------------------+------------------+
 count| 1501| 1501| 1501|
 mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
 stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
 min| 0| 1.0| 0|
 max| 99| 5.0| 29|
+-------+------------------+------------------+------------------+

In [9]:
training, test = data.randomSplit([0.8, 0.2])

In [10]:
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

In [11]:
model = als.fit(training)

In [12]:
predictions = model.transform(test)

In [13]:
predictions.show()

+-------+------+------+-------------+
movieId|rating|userId| prediction|
+-------+------+------+-------------+
 31| 1.0| 27| 1.2053841|
 31| 4.0| 12| 1.1060678|
 31| 3.0| 8|-0.0017251521|
 31| 3.0| 14| -1.3511903|
 85| 1.0| 13| 1.9975404|
 85| 5.0| 8| -0.28216773|
 85| 1.0| 2| -3.7590122|
 65| 1.0| 28| 1.2900844|
 53| 1.0| 7| 2.5262313|
 53| 3.0| 14| 0.3678335|
 78| 1.0| 27| 0.5861535|
 78| 1.0| 20| 1.1188818|
 78| 1.0| 11| 1.4991599|
 34| 3.0| 3| -0.6317695|
 34| 1.0| 19| -0.06309592|
 81| 1.0| 3| 0.74811816|
 81| 1.0| 19| 0.5089559|
 81| 4.0| 11| 2.5179715|
 28| 1.0| 5| 2.5657134|
 76| 1.0| 19| 1.8127615|
+-------+------+------+-------------+
only showing top 20 rows

In [14]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [15]:
rmse = evaluator.evaluate(predictions)

In [16]:
print('RMSE')
print(rmse)

RMSE
1.7092854132808888

In [17]:
single_user = test.filter(test['userId']==11).select(['movieId', 'userId'])
single_user.show()

+-------+------+
movieId|userId|
+-------+------+
 10| 11|
 12| 11|
 20| 11|
 36| 11|
 47| 11|
 61| 11|
 62| 11|
 66| 11|
 72| 11|
 78| 11|
 79| 11|
 80| 11|
 81| 11|
 88| 11|
 97| 11|
+-------+------+

In [18]:
recommendations = model.transform(single_user)

In [19]:
recommendations.orderBy('prediction', ascending=False).show()

+-------+------+-----------+
movieId|userId| prediction|
+-------+------+-----------+
 66| 11| 4.284766|
 62| 11| 3.5991716|
 81| 11| 2.5179715|
 80| 11| 2.466096|
 10| 11| 2.0533426|
 79| 11| 2.0521886|
 88| 11| 2.0430758|
 36| 11| 1.8171954|
 78| 11| 1.4991599|
 20| 11| 1.2098188|
 47| 11| 0.29576194|
 61| 11| 0.26863858|
 12| 11|-0.25631368|
 72| 11| -0.5220659|
 97| 11| -0.8735642|
+-------+------+-----------+